In [1]:
!pip install mp-api --upgrade
import numpy as np
from mp_api.client import MPRester
#from pymatgen.ext.matproj import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

No module named 'phonopy'
No module named 'phonopy'


ModuleNotFoundError: No module named 'jupyter_jsmol'

In [ ]:
mp_key = "XJhzTsEnBSh1B3Uh2ARw9cKQZuE9Q90J"
mp_id = "mp-569779"

In [ ]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

# Structure symmetry

In [ ]:
i_atom = 7
i_symmop = 2

view = quick_view(prim_struc, "packed", conventional = True)
display(view)
view.script('select antimony; color lightsalmon')
view.script('select sodium; color gold')
view.script('select lithium; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

In [ ]:
symmop = symmops[i_symmop - 1]
print(symmop)

In [ ]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print(pos_init)

In [ ]:
pos_final = symmop.operate(pos_init)
print(pos_final)

In [ ]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print(j_atom, conv_struc.sites[j_atom -1].frac_coords)

# Brillouin zone

In [ ]:
plot_brillouin_zone(el_bs.structure)

# Electronic bandstructure

In [ ]:
fig_el_bs = get_plot_bs(el_bs, plot_range=[-4,7])
fig_el_bs.show()

In [ ]:
fig_el_dos = get_plot_dos(el_dos, plot_range=[-4,7])
fig_el_dos.show()

In [ ]:
fig_el_bs_and_dos = get_plot_bs_and_dos(el_bs, el_dos, plot_range=[-4,7])
fig_el_bs_and_dos.show()

In [ ]:
xvals = fig_el_bs.to_dict()["data"][0]["x"]
yvals_vbm = fig_el_bs.to_dict()["data"][2]["y"]
yvals_cbm = fig_el_bs.to_dict()["data"][3]["y"]

In [ ]:
xvals_band_edges = []
yvals_band_edges = []
for i in el_bs.get_vbm()["kpoint_index"]:
    xvals_band_edges.append(xvals[i])
    yvals_band_edges.append(yvals_vbm[i])
for i in el_bs.get_cbm()["kpoint_index"]:
    xvals_band_edges.append(xvals[i])
    yvals_band_edges.append(yvals_cbm[i])

In [ ]:
for i in el_bs.get_vbm()["kpoint_index"]:
    scatter = go.Scatter(
        x = xvals_band_edges, y = yvals_band_edges,
        mode = "markers", marker = dict(color="black"),
        showlegend=False)
    fig_el_bs.add_trace(scatter)
fig_el_bs.update_layout(xaxis_range = [xvals[0], xvals[-1]])
fig_el_bs.show()

In [ ]:
el_bs.get_vbm()

In [ ]:
el_bs.get_cbm()

In [ ]:
i_vbm = list(el_bs.get_vbm()['band_index'].values())[-1][-1]
i_cbm = list(el_bs.get_cbm()['band_index'].values())[-1][0]
print("VBM:", i_vbm, "CBM:", i_cbm)

# Phonon bandstructure

In [ ]:
fig_ph_bs = get_plot_bs(ph_bs)
fig_ph_bs.update_yaxes(rangemode="tozero")
fig_ph_bs.show()

In [ ]:
fig_ph_dos = get_plot_dos(ph_dos)
fig_ph_dos.show()

In [ ]:
fig_ph_bs_and_dos = get_plot_bs_and_dos(ph_bs, ph_dos)
fig_ph_bs_and_dos.update_yaxes(rangemode="tozero")
fig_ph_bs_and_dos.show()

# Distances in the Brillouin zone

In [ ]:
xvals = fig_el_bs.to_dict()["data"][0]["x"]
for i_branch, branch in enumerate(el_bs.branches):
    wavevectors = get_branch_wavevectors(el_bs, i_branch)
    v0 = np.dot(np.transpose(el_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[0])
    v1 = np.dot(np.transpose(el_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[1])
    print("{0:3d} {1:9.6f} {2:9.6f}".format(i_branch, np.linalg.norm(v1-v0), xvals[branch["end_index"]]-xvals[branch["start_index"]]))

In [ ]:
xvals = fig_ph_bs.to_dict()["data"][0]["x"]
for i_branch, branch in enumerate(ph_bs.branches):
    wavevectors = get_branch_wavevectors(ph_bs, i_branch)
    v0 = np.dot(np.transpose(ph_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[0])
    v1 = np.dot(np.transpose(ph_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[1])
    print("{0:3d} {1:9.6f} {2:9.6f}".format(i_branch, np.linalg.norm(v1-v0), xvals[branch["end_index"]]-xvals[branch["start_index"]]))

In [ ]:
el_bs.structure

In [ ]:
ph_bs.structure

In [ ]:
prim_struc

# Specific heat

In [ ]:
temperatures = np.arange(0,1000,5)
R = 8.314
nat = len(prim_struc)
ph_cv = np.array([ph_dos.cv(temperatures[i]) for i in range(len(temperatures))])/(3*nat*R)

In [ ]:
fig = go.Figure()
scatter = go.Scatter(x=temperatures, y=ph_cv)
fig.add_trace(scatter)
fig.add_hline(y=1, line_width=2, line_color="red")
fig.update_layout(
    xaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
    yaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
    xaxis_title = "Temperature",
    yaxis_title = "$C_v\,/\,3N_{\!at}R$",        
)
fig.show()